In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import groupby
import statistics
import copy
from langdetect import detect

In [2]:
df = pd.read_csv('../data/blogtext.csv')
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [3]:
authors = df['id'].values

print('Amount of blog posts:',len(df))
print('Number of authors:',len(set(authors)))

Amount of blog posts: 681284
Number of authors: 19320


In [4]:
min_blogs = 100
min_blog_median = 1000

unique_authors = [a for a in set(authors)]
medians = []

for a in unique_authors:
    a_df = df[df['id'] == a]
    blogs = len(a_df)
    if blogs >= min_blogs:
        median_tuple = (statistics.median([len(t) for t in a_df['text'].values]),blogs,a)
        if median_tuple[0] >= min_blog_median:
            medians.append(median_tuple)
            
medians = sorted(medians, reverse=True)

candidate_auths = [p[2] for p in medians]
df_top = df[df['id'].map(lambda i: i in candidate_auths)]

print('Authors left after selection:',len(medians))
print('Number of blogs in selected dataset:',len(df_top))

Authors left after selection: 338
Number of blogs in selected dataset: 69867


In [5]:
selected_auths = []

for a in candidate_auths:
    a_df = df_top[df_top['id'] == a]
    blogs = a_df['text'].values
    
    english = True
    for b in blogs:
        if len(b) > 250:
            if detect(b) != 'en':
                english = False
    
    if english:
        selected_auths.append(a)

print('Remaining authors after language selection:', len(selected_auths))

Remaining authors after language selection: 304


In [6]:
df_selected = df_top[df_top['id'].map(lambda i: i in selected_auths)]
df_selected.to_csv('../data/blogtext_selection.csv')